# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


## Objectives



*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities


Let's first import required Python packages for this lab:


In [33]:
%pip install folium 
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [34]:
import folium
import pandas as pd

In [35]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [36]:
import requests
import io
import pandas as pd

# URL of the CSV file
URL1 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Fetch the data from the URL
response = requests.get(URL1)

# Read the CSV file from the response content
spacex_df = pd.read_csv(io.StringIO(response.text))

#### Coordinates for each site

In [37]:
# Selected relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites.  Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [38]:
# initiates coordinates for NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [39]:
# Created a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Created a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Created an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`


In [40]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, added a Circle object based on its coordinate (Lat, Long) values. In addition, added Launch site name as a popup label
for index, row in launch_sites_df.iterrows():
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=50,  # Radius in meters (5 km in this case)
        color='blue',
       fill_color='blue', 
        fill_opacity=0.1
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    
    folium.map.Marker(
        location=[row['Lat'], row['Long']],icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 32; font-weight: bold; color:#8B0A1A;"><b>%s</b></div>' % row['Launch Site'],
    )).add_to(site_map)

# Added a line for the equator
folium.PolyLine([(0, -180), (0, 180)], color='red', weight=2.5, opacity=1, tooltip="Equator").add_to(site_map)

# Added a line for the Tropic of Cancer
folium.PolyLine([(23.5, -180), (23.5, 180)], color='blue', weight=2.5, opacity=1, tooltip="Tropic of Cancer").add_to(site_map)

# Added a line for the Tropic of Capricorn
folium.PolyLine([(-23.5, -180), (-23.5, 180)], color='green', weight=2.5, opacity=1, tooltip="Tropic of Capricorn").add_to(site_map)
site_map.save("world_map.html")

#  map displayed
site_map

### Task 2: success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [41]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [42]:
marker_cluster = MarkerCluster()

Created a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [43]:
# Applied a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

def map_class_to_color(class_value):
    if class_value == 1:
        return 'green'
    elif class_value == 0:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(map_class_to_color)

For each launch result in `spacex_df` data frame, added a `folium.Marker` to `marker_cluster`


In [44]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [45]:
# Added marker_cluster to current site_map
site_map.add_child(marker_cluster)


# Created a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Iterated over rows and add markers to the cluster
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color='white', icon_color=row['marker_color']),
        popup=f"Launch Site: {row['Launch Site']}\nClass: {row['class']}"
    ).add_to(marker_cluster)

site_map

From the color-labeled markers in marker clusters, It shall be easy to identify which launch sites have relatively high success rates.


### TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map so that while exploring the map, one can easily find the coordinates of any points of interests (such as railway).

In [46]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [47]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Marking down a point on the closest coastline using MousePosition and calculated the distance between the coastline point and the launch site.


In [48]:
coastline_Lat = 28.56283
coastline_Lon = -80.56790
launch_site_lat = 28.562302	
launch_site_lon = -80.577356
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_Lat, coastline_Lon)

In [49]:
# Created and added a folium.Marker on  selected closest coastline point on the map
# Displayed the distance between coastline point and launch site using the icon property 

folium.Marker( location=[row['Lat'], row['Long']]).add_to(site_map)

#  marker for closest coastline point with distance information
folium.Marker(
   location=(coastline_Lat,coastline_Lon),
    icon=folium.DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
).add_to(site_map)

Draw a `PolyLine` between a launch site to the selected coastline point


In [50]:
# Created a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coastline_point=[28.56283,-80.56790]
lines=folium.PolyLine(
        locations=[coastline_point, [row['Lat'], row['Long']]],
        weight=1
    ).add_to(site_map)
site_map.add_child(lines)

In [51]:
# coordinates of the nearest railway
rail_Lat = 28.57115
rail_Lon = -80.58543
distance_rail = calculate_distance(launch_site_lat, launch_site_lon, rail_Lat, rail_Lon)


folium.Marker( location=[row['Lat'], row['Long']]).add_to(site_map)

# Adding marker for closest coastline point with distance information
folium.Marker(
   location=(rail_Lat, rail_Lon),
    icon=folium.DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_rail),
    )
).add_to(site_map)

# Created a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
railway_point=[28.57115,-80.58543]
Rlines=folium.PolyLine(
        locations=[railway_point, [row['Lat'], row['Long']]],
        weight=1
    ).add_to(site_map)
site_map.add_child(Rlines)

In [52]:
# coordinates of the nearest highway
high_Lat = 28.56287
high_Lon = -80.5708
distance_hw = calculate_distance(launch_site_lat, launch_site_lon, high_Lat, high_Lon)


folium.Marker( location=[row['Lat'], row['Long']]).add_to(site_map)

# Add marker for closest highway point with distance information
folium.Marker(
   location=(high_Lat, high_Lon),
    icon=folium.DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_hw),
    )
).add_to(site_map)

# Created a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
high_point=[28.56287,-80.5708]
hlines=folium.PolyLine(
        locations=[high_point, [row['Lat'], row['Long']]],
        weight=1
    ).add_to(site_map)
site_map.add_child(hlines)

In [53]:
# coordinates of the most nearest city
city_Lat = 28.61197
city_Lon = -80.80743
distance_c = calculate_distance(launch_site_lat, launch_site_lon, city_Lat, city_Lon)


folium.Marker( location=[row['Lat'], row['Long']]).add_to(site_map)

# Add marker for closest highway point with distance information
folium.Marker(
   location=(city_Lat, city_Lon),
    icon=folium.DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_c),
    )
).add_to(site_map)

# Created a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
city_point=[28.61197,-80.80743]
clines=folium.PolyLine(
        locations=[city_point, [row['Lat'], row['Long']]],
        weight=1
    ).add_to(site_map)
site_map.add_child(clines)

In [54]:
# coordinates of the most nearest city
TC_Lat = 23.5
TC_Lon = -80.29143
distance_T = calculate_distance(launch_site_lat, launch_site_lon, TC_Lat, TC_Lon)


folium.Marker( location=[row['Lat'], row['Long']]).add_to(site_map)

# Add marker for closest highway point with distance information
folium.Marker(
   location=(TC_Lat, TC_Lon),
    icon=folium.DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_c),
    )
).add_to(site_map)

# Created a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
TC_point=[23.5,-80.29143]
TClines=folium.PolyLine(
        locations=[TC_point, [row['Lat'], row['Long']]],
        weight=1
    ).add_to(site_map)
site_map.add_child(TClines)

site_map

**INSIGHTS GATHERED**

*  CCAFS SLC 40 is 1.26 km away from nearest railways.
*  CCAFS SLC 40 is 1 km away from nearest highways.
*  CCAFS SLC 40 is 1.26 km away from nearest coastline. 
*  CCAFS SLC 40 is 23.14 km  away from nearest city. 
*  CCAFS SLC 40 is 23.14 km  away from Tropic of Cancer.
*  Launch sites should be far from nearby cities.
